In [41]:
from __future__ import print_function

import pandas as pd
import numpy as np
import ipywidgets

from ipyleaflet import *

In [42]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.colors

def n_colors(n, colormap=mpl.cm.Blues):
    data = np.linspace(0.0,1.0,n)
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(data)]
    return c

def data_to_colors(data, colormap=mpl.cm.plasma):
    c = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(mpl.colors.Normalize()(data))]
    return c

In [43]:
m1 = Map(center=[37.7749, -122.34580993652344], zoom=12, layout=Layout(height='600px'))

In [44]:
survey_data = pd.read_csv('./../analysis_data/scores_by_zipcodes.csv', header=None)

In [45]:
colors = data_to_colors(survey_data.ix[:, 1], colormap=mpl.cm.Greens)

In [46]:
zips = [str(int(i)) for i in survey_data.ix[:, 0]]

In [47]:
cols = {}
for i in range(len(zips)):
    cols[zips[i]] = colors[i]

In [48]:
import json
with open('./../mapdata/sf_zipcodes.geojson') as f:
    data = json.load(f)

In [49]:
for feature in data['features']:
    feature['properties']['style'] = {
        'color': cols[feature['id']],
        'weight': 1,
        'fillColor': cols[feature['id']],
        'fillOpacity': 0.75,
    }

In [50]:
g = GeoJSON(data=data)
m1 += g

In [51]:
m1

In [52]:
m = Map(center=[37.7749, -122.34580993652344], zoom=12, layout=Layout(height='600px'))

In [53]:
tract_data = pd.read_csv('./../data/CensusData/sfo data/tract_data_normalized.csv')

In [54]:
colors = data_to_colors(tract_data['pct_bachelors'], colormap=mpl.cm.Reds)

In [55]:
tracts = [str(int(i)) for i in tract_data.ix[:, 0].values]

In [56]:
cols = {}
for i in range(len(tracts)):
    cols[tracts[i][4:]] = colors[i]

In [57]:
import json
with open('./../mapdata/sf_tracts.geojson') as f:
    data = json.load(f)

In [58]:
for feature in data['features']:
    try:
        style_col = cols[feature['properties']['tractce10']]
    except KeyError:
        style_col = 'Grey'
    feature['properties']['style'] = {
        'color': style_col,
        'weight': 1,
        'fillColor': style_col,
        'fillOpacity':0.75,
    }

In [59]:
g = GeoJSON(data=data)
m += g

In [60]:
from ipywidgets import Dropdown, VBox

In [61]:
from IPython.display import display

In [62]:
data_columns = ['pct_bachelors', 'labor_part_rate', 'pct_welfare', 'low_pov_idx',
      'labor_idx', 'env_health_idx', 'hispanic', 'white', 'black',
      'american_indian', 'asian', 'pac_islander', 'other_races',
      'two_races', 'housing_cost', 'transportation_cost',
      'pub_school_score', 'pr_school_score', 'rest_score',
      'rest_proximity']

x = Dropdown(options=data_columns, value='pct_bachelors')

In [63]:
def on_changed(name, value):
    #if len(m.layers) > 0:
    #    m.remove_layer(m.layers[-1])
    colors = data_to_colors(tract_data[x.value], colormap=mpl.cm.Reds)
    cols = {}
    for i in range(len(tracts)):
        cols[tracts[i][4:]] = colors[i]
    for feature in data['features']:
        try:
            style_col = cols[feature['properties']['tractce10']]
        except KeyError:
            style_col = 'grey'
        feature['properties']['style'] = {
            'color': style_col,
            'weight': 1,
            'fillColor': style_col,
            'fillOpacity':0.75,
        }
    g = GeoJSON(data=data)
    m.layers = [m.layers[0], g]
    
x.on_trait_change(on_changed, 'value')

/home/sylvain/focus/ipython/ipykernel/ipykernel/__main__.py:22: DeprecationWarning: on_trait_change is deprecated: use observe instead


In [67]:
m.layout.align_self = 'stretch'
display(VBox([m, x]))